In [1]:
!pip show tensorflow tensorflow-probability
!pip install tensorflow-probability==0.24.0
!pip install --upgrade keras-tuner
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.visualizations.git
!pip install mne #The MNE Package is installed
FILEID = "1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip -y #Package with useful functions for motor imagery classification based in EEG.
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.EEG_Tensorflow_models.git
!dir

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf_keras
---
Name: tensorflow-probability
Version: 0.25.0
Summary: Probabilistic modeling and statistical inference in TensorFlow
Home-page: http://github.com/tensorflow/probability
Author: Google LLC
Author-email: no-reply@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, cloudpickle, decorator, dm-tree, gast, numpy, six
Required-by: dopamine_r

In [2]:
!pip install --upgrade botocore tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 94.4 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.38.11
    Uninstalling botocore-1.38.11:
      Successfully uninstalled botocore-1.38.11
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.38.11 requires botocore<1.39.0,>=1.38.11, but you have botocore 1.40.0 which is incompatible.
tensorflow 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.20.0 which is incompatible.


In [5]:
import numpy as np
import os
import itertools
import random
import pickle
import mne
import h5py
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import keras_tuner as kt


from gcpds.databases.BCI_Competition_IV import Dataset_2a
from typing import Sequence, Tuple
from scipy.signal import iirnotch, filtfilt, butter, freqz
import matplotlib.pyplot as plt
from scipy.spatial import distance
import networkx as nx
from tqdm import tqdm
from mne.preprocessing import compute_current_source_density
from mne.channels import make_standard_montage, read_custom_montage
from scipy.signal import butter, filtfilt, resample, iirnotch
from gcpds.visualizations.series import plot_eeg
from scipy.stats import norm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from tqdm import tqdm

from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from gcpds.databases import GIGA_MI_ME
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import LeavePGroupsOut, StratifiedGroupKFold
from tensorflow.keras.models import Model
from scipy.spatial.distance import cdist
from sklearn.model_selection import GroupKFold
from tensorflow.keras.models import load_model

from keras_tuner import Objective
from keras_tuner import HyperModel
from keras.layers import Layer, Activation
from keras_tuner import BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.metrics import BinaryAccuracy, Recall

In [6]:
def load_BCICIV2a(db, sbj: int, mode: str, fs: float) -> tuple:
    """
    Carga los datos EEG para un sujeto específico con preprocesamiento, incluyendo filtrado y laplaciano superficial.
    Se recorta el tiempo entre los segundos 2 y 4.
    
    Args:
        db (Dataset_2a): Objeto del dataset.
        sbj (int): Identificador del sujeto (1-9).
        mode (str): 'training' o 'testing'.
        fs (float): Frecuencia de muestreo.

    Returns:
        tuple: Datos EEG preprocesados con laplaciano superficial y recortados en el tiempo (X), etiquetas (y).
    """
    # Cargar los datos del sujeto
    db.load_subject(sbj, mode=mode)
    X, y = db.get_data()  # Datos y etiquetas
    X = X[:, :-3, :]  # Seleccionar solo los canales EEG (22 canales)
    X = X * 1e6  # Convertir a microvoltios

    # Aplicar filtro Notch (50 Hz)
    notch_freq = 50.0
    q_factor = 30.0
    b_notch, a_notch = iirnotch(w0=notch_freq, Q=q_factor, fs=fs)
    X = filtfilt(b_notch, a_notch, X, axis=2)

    # Aplicar filtro pasa banda (0.5 - 100 Hz)
    lowcut = 0.5
    highcut = 100.0
    b_band, a_band = butter(N=4, Wn=[lowcut, highcut], btype='bandpass', fs=fs)
    X = filtfilt(b_band, a_band, X, axis=2)

    # Recortar los datos entre el segundo 2 y el segundo 4
    start_sample = int(2 * fs)  # Inicio en el segundo 2
    end_sample = int(4 * fs)    # Fin en el segundo 4
    X = X[:, :, start_sample:end_sample]

    # Lista de nombres de los 22 canales EEG (sin EOG)
    eeg_channel_names = [
        'Fz', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4',
        'C6', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'P1', 'Pz', 'P2', 'POz'
    ]

    # Crear información para los canales EEG
    info = mne.create_info(
        ch_names=eeg_channel_names,
        sfreq=fs,  # Usar la frecuencia original
        ch_types=["eeg"] * len(eeg_channel_names)  # Todos los canales son EEG
    )

    # Cargar un montaje estándar basado en el sistema 10/20
    montage = mne.channels.make_standard_montage('standard_1020')
    info.set_montage(montage)

    # Aplicar el cálculo del laplaciano superficial 
    laplacian_X = []
    for trial in X:
        # Crear un objeto RawArray para cada prueba
        raw = mne.io.RawArray(trial, info)
        # Calcular el laplaciano superficial
        raw = mne.preprocessing.compute_current_source_density(raw)
        # Obtener los datos con el laplaciano aplicado
        laplacian_X.append(raw.get_data())

    # Reconvertir a un array numpy con la misma forma original
    X = np.stack(laplacian_X)

    return X, y

In [7]:
def load_BCICIV2a(db, sbj: int, mode: str, fs: float) -> tuple:
    """
    Carga los datos EEG para un sujeto específico con preprocesamiento, incluyendo filtrado y laplaciano superficial.
    Se recorta el tiempo entre los segundos 2 y 4 y solo se incluyen las clases 'mano izquierda' (1) y 'mano derecha' (2).
    
    Args:
        db (Dataset_2a): Objeto del dataset.
        sbj (int): Identificador del sujeto (1-9).
        mode (str): 'training' o 'testing'.
        fs (float): Frecuencia de muestreo.

    Returns:
        tuple: Datos EEG preprocesados con laplaciano superficial y recortados en el tiempo (X), etiquetas (y).
    """
    # Cargar los datos del sujeto
    db.load_subject(sbj, mode=mode)
    X, y = db.get_data()  # Datos y etiquetas
    X = X[:, :-3, :]  # Seleccionar solo los canales EEG (22 canales)
    X = X * 1e6  # Convertir a microvoltios

    # Aplicar filtro Notch (50 Hz)
    notch_freq = 50.0
    q_factor = 30.0
    b_notch, a_notch = iirnotch(w0=notch_freq, Q=q_factor, fs=fs)
    X = filtfilt(b_notch, a_notch, X, axis=2)

    # Aplicar filtro pasa banda (0.5 - 100 Hz)
    lowcut = 0.5
    highcut = 100.0
    b_band, a_band = butter(N=4, Wn=[lowcut, highcut], btype='bandpass', fs=fs)
    X = filtfilt(b_band, a_band, X, axis=2)

    # Recortar los datos entre el segundo 2 y el segundo 4
    start_sample = int(2 * fs)  # Inicio en el segundo 2
    end_sample = int(4 * fs)    # Fin en el segundo 4
    X = X[:, :, start_sample:end_sample]

    # Filtrar solo las clases de interés (1: mano izquierda, 2: mano derecha)
    clases = [0, 1]
    mask = np.isin(y, clases)
    X = X[mask]
    y = y[mask]

    # Lista de nombres de los 22 canales EEG (sin EOG)
    eeg_channel_names = [
        'Fz', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4',
        'C6', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'P1', 'Pz', 'P2', 'POz'
    ]

    # Crear información para los canales EEG
    info = mne.create_info(
        ch_names=eeg_channel_names,
        sfreq=fs,  # Usar la frecuencia original
        ch_types=["eeg"] * len(eeg_channel_names)  # Todos los canales son EEG
    )

    # Cargar un montaje estándar basado en el sistema 10/20
    montage = mne.channels.make_standard_montage('standard_1020')
    info.set_montage(montage)

    # Aplicar el cálculo del laplaciano superficial 
    laplacian_X = []
    for trial in X:
        # Crear un objeto RawArray para cada prueba
        raw = mne.io.RawArray(trial, info)
        # Calcular el laplaciano superficial
        raw = mne.preprocessing.compute_current_source_density(raw)
        # Obtener los datos con el laplaciano aplicado
        laplacian_X.append(raw.get_data())

    # Reconvertir a un array numpy con la misma forma original
    X = np.stack(laplacian_X)

    return X, y

In [8]:
@register_keras_serializable(package="CustomLayers")
class TakensConv1D(tf.keras.layers.Layer):
    def __init__(self, dx=5, dy=5, tau=1, mu=4, **kwargs):
        super().__init__(**kwargs)
        self.dx = int(dx)
        self.dy = int(dy)
        self.tau = int(tau)
        self.mu = int(mu)
        self.num_filters = self.dx + self.dy + 1

    def build(self, input_shape):
        kernel_size = self.mu + (self.dx - 1) * self.tau + 1
        kernel_shape = (kernel_size, 1, self.num_filters)
        kernel = tf.zeros(kernel_shape, dtype=tf.float32)

        offsets_x = self.mu + tf.range(self.dx) * self.tau
        offsets_y = tf.range(1, self.dy + 1) * self.tau
        offset_y_t = tf.constant([0], dtype=tf.int32)

        filas_x = tf.range(self.dx)
        filas_y = self.dx + tf.range(self.dy)
        fila_y_t = tf.constant([self.dx + self.dy])

        cols_x = filas_x
        cols_y = filas_y
        col_y_t = fila_y_t

        idx_x = tf.stack([filas_x, offsets_x, cols_x], axis=1)
        idx_y = tf.stack([filas_y, offsets_y, cols_y], axis=1)
        idx_yt = tf.stack([fila_y_t, offset_y_t, col_y_t], axis=1)

        indices_total = tf.concat([idx_x, idx_y, idx_yt], axis=0)
        updates = tf.ones([tf.shape(indices_total)[0]], dtype=tf.float32)

        sort_order = tf.argsort(indices_total[:, 0])
        indices_sorted = tf.gather(indices_total, sort_order)

        row_for_scatter = tf.cast(indices_sorted[:, 1], tf.int32)
        col_for_scatter = tf.cast(indices_sorted[:, 2], tf.int32)

        final_indices = tf.stack([row_for_scatter,
                                  tf.zeros_like(row_for_scatter),
                                  col_for_scatter], axis=1)

        kernel = tf.scatter_nd(final_indices, updates, kernel_shape)
        self.kernel = kernel.numpy()[::-1]

        self.conv1d = tf.keras.layers.Conv1D(
            filters=self.num_filters,
            kernel_size=kernel_size,
            strides=self.tau,
            padding="valid",
            use_bias=False
        )
        self.conv1d.build((None, input_shape[2], 1))
        self.conv1d.set_weights([self.kernel])
        self.conv1d.trainable=False

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        channels = tf.shape(inputs)[1]
        time_steps = tf.shape(inputs)[2]

        reshaped = tf.reshape(inputs, (-1, time_steps, 1))
        conv_output = self.conv1d(reshaped)
        new_time = tf.shape(conv_output)[1]

        output = tf.reshape(conv_output, 
                            (batch_size, channels, new_time, self.num_filters))

        x_sub_t_minus_mu = output[..., :self.dx]
        y_sub_t_minus_1 = output[..., self.dx:self.dx + self.dy]
        y_sub_t = output[..., -1:]

        return x_sub_t_minus_mu, y_sub_t_minus_1, y_sub_t
       

@register_keras_serializable(package="CustomLayers")
class KernelLayer(tf.keras.layers.Layer):
    def __init__(self, 
                 amplitude=1.0,
                 trainable_amplitude=False, 
                 length_scale=1.0,
                 trainable_length_scale=False,
                 alpha=1.0,  # Solo usado para Rational Quadratic
                 trainable_alpha=False,
                 kernel_type="gaussian",  # "gaussian" o "rational_quadratic"
                 **kwargs):
        super(KernelLayer, self).__init__(**kwargs)

        self.init_amplitude = amplitude
        self.trainable_amplitude = trainable_amplitude
        self.init_length_scale = length_scale
        self.trainable_length_scale = trainable_length_scale
        self.init_alpha = alpha
        self.trainable_alpha = trainable_alpha
        self.kernel_type = kernel_type.lower()

    def build(self, input_shape):
        self.amplitude = self.add_weight(
            name="amplitude",
            shape=(),
            initializer=tf.constant_initializer(self.init_amplitude),
            trainable=self.trainable_amplitude,
            dtype=self.dtype
        )

        self.length_scale = self.add_weight(
            name="length_scale",
            shape=(),
            initializer=tf.constant_initializer(self.init_length_scale),
            trainable=self.trainable_length_scale,
            dtype=self.dtype
        )

        if self.kernel_type == "rational_quadratic":
            self.alpha = self.add_weight(
                name="alpha",
                shape=(),
                initializer=tf.constant_initializer(self.init_alpha),
                trainable=self.trainable_alpha,
                dtype=self.dtype
            )

        super(KernelLayer, self).build(input_shape)

    def call(self, X):
        if self.kernel_type == "gaussian":
            kernel = tfp.math.psd_kernels.ExponentiatedQuadratic(
                amplitude=self.amplitude,
                length_scale=self.length_scale
            )
        elif self.kernel_type == "rational_quadratic":
            kernel = tfp.math.psd_kernels.RationalQuadratic(
                amplitude=self.amplitude,
                length_scale=self.length_scale,
                scale_mixture_rate=self.alpha
            )
        else:
            raise ValueError(f"Unsupported kernel_type: {self.kernel_type}")
        
        return kernel.matrix(X, X)
       
@register_keras_serializable(package="CustomLayers")
class TransferEntropyLayer(tf.keras.layers.Layer):
    def __init__(self, alpha=2, **kwargs):

        super().__init__(**kwargs)
        self.alpha = int(alpha)

    def compute_entropy(self, K_hadamard):
        
        trace_hadamard = tf.reduce_sum(tf.linalg.diag_part(K_hadamard), axis=-1) 
        trace_hadamard = tf.expand_dims(tf.expand_dims(trace_hadamard, axis=-1), axis=-1)
        K_normalized = K_hadamard / trace_hadamard

        K_power = tf.linalg.matmul( K_normalized , K_normalized, grad_a=True, grad_b=True ) 
        trace_power = tf.reduce_sum(tf.linalg.diag_part(K_power), axis=-1)  
        H_alpha = (1 / (1 - self.alpha)) * tf.math.log(trace_power)
        return H_alpha

    def call(self, K_x, K_y_minus_1, K_y):
        
        K_x_exp = tf.expand_dims(K_x, axis=2)
        
        K_y_minus_1_exp = tf.expand_dims(K_y_minus_1, axis=1)
        K_y_exp = tf.expand_dims(K_y, axis=1)

        
        H_1 = self.compute_entropy(K_y_minus_1_exp * K_x_exp)
        H_2 = self.compute_entropy(K_y_exp * K_y_minus_1_exp * K_x_exp)
        H_3 = self.compute_entropy(K_y_exp * K_y_minus_1_exp)
        H_4 = self.compute_entropy(K_y_minus_1_exp)

        TE = H_1 - H_2 + H_3 - H_4
        
        return TE
        
@register_keras_serializable(package="CustomLayers")      
class RemoveDiagonalFlatten(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
    
        super().__init__(**kwargs)

    def call(self, inputs):
        
        shape_dyn = tf.shape(inputs)  
        batch_size = shape_dyn[0]
        c = tf.shape(inputs)[1]

        
        tf.debugging.assert_equal(
            tf.shape(inputs)[1], tf.shape(inputs)[2],
            message="RemoveDiagonalFlatten: la matriz de entrada no es cuadrada."
        )  
        diag_mask = tf.eye(c, dtype=inputs.dtype)  
        inputs_no_diag = inputs * (1 - diag_mask) 
        flattened = tf.reshape(inputs_no_diag, [batch_size, -1])  
        non_diag = tf.boolean_mask(flattened, tf.reshape(1 - diag_mask, [-1]), axis=1)
        num_features = c * (c - 1)  
        result = tf.reshape(non_diag, [batch_size, num_features])  

        return result

In [9]:
from tensorflow.keras.metrics import Metric
@register_keras_serializable(package="CustomMetrics")
class Sensitivity(Metric):
    def __init__(self, name='sensitivity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = tf.keras.metrics.TruePositives()
        self.fn = tf.keras.metrics.FalseNegatives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tp.update_state(y_true, y_pred, sample_weight)
        self.fn.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.tp.result() / (self.tp.result() + self.fn.result() + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.reset_states()
        self.fn.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config


@register_keras_serializable(package="CustomMetrics")
class Accuracy(Metric):
    def __init__(self, name='accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = tf.keras.metrics.TruePositives()
        self.tn = tf.keras.metrics.TrueNegatives()
        self.fp = tf.keras.metrics.FalsePositives()
        self.fn = tf.keras.metrics.FalseNegatives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tp.update_state(y_true, y_pred, sample_weight)
        self.tn.update_state(y_true, y_pred, sample_weight)
        self.fp.update_state(y_true, y_pred, sample_weight)
        self.fn.update_state(y_true, y_pred, sample_weight)

    def result(self):
        num = self.tp.result() + self.tn.result()
        den = num + self.fp.result() + self.fn.result() + tf.keras.backend.epsilon()
        return num / den

    def reset_states(self):
        self.tp.reset_states()
        self.tn.reset_states()
        self.fp.reset_states()
        self.fn.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config


@register_keras_serializable(package="CustomMetrics")
class F1Score(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = tf.keras.metrics.TruePositives()
        self.fp = tf.keras.metrics.FalsePositives()
        self.fn = tf.keras.metrics.FalseNegatives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tp.update_state(y_true, y_pred, sample_weight)
        self.fp.update_state(y_true, y_pred, sample_weight)
        self.fn.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.tp.result() / (self.tp.result() + self.fp.result() + tf.keras.backend.epsilon())
        recall = self.tp.result() / (self.tp.result() + self.fn.result() + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tp.reset_states()
        self.fp.reset_states()
        self.fn.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config


@register_keras_serializable(package="CustomMetrics")
class Specificity(Metric):
    def __init__(self, name='specificity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tn = tf.keras.metrics.TrueNegatives()
        self.fp = tf.keras.metrics.FalsePositives()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.tn.update_state(y_true, y_pred, sample_weight)
        self.fp.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.tn.result() / (self.tn.result() + self.fp.result() + tf.keras.backend.epsilon())

    def reset_states(self):
        self.tn.reset_states()
        self.fp.reset_states()

    def get_config(self):
        base_config = super().get_config()
        return base_config

In [10]:
import botocore, tensorboard, urllib3
print("botocore:", botocore.__version__)
print("tensorboard:", tensorboard.__version__)
print("urllib3:", urllib3.__version__)

botocore: 1.40.0
tensorboard: 2.20.0
urllib3: 1.26.20


In [11]:
# Crear instancia del dataset
db = Dataset_2a('/kaggle/input/dataset-2a')
fs = 250.0 

# Cargar los datos del sujeto en modo 'training'
X, y = load_BCICIV2a(db, sbj=4, mode='training', fs=fs)
print(f'tamaño de X:', X.shape)
print(f'tamaño de y:', y.shape)

tamaño de X: (129, 22, 500)
tamaño de y: (129,)


In [12]:
train_data, val_data, train_labels, val_labels = train_test_split(
    X, y,
    test_size=0.2,
    random_state=72,
    stratify=y
)

def create_tunable_model(dx, dy, tau, mu,
                         kernel_type, learning_rate,
                         kernel_size,
                         dropout_rate): 
    input_eeg = tf.keras.Input(shape=(22, 500), name='input_eeg')

    x = tf.keras.layers.DepthwiseConv1D(
        kernel_size=kernel_size,
        strides=1,
        padding='valid',
        activation='relu',
        depth_multiplier=1,
        data_format="channels_first",
        name='block1_depthwise_conv1d'
    )(input_eeg)

    x = tf.keras.layers.AveragePooling1D(
        pool_size=4,
        strides=4,
        padding='valid',
        data_format="channels_first",
        name='block1_avg_pooling'
    )(x)

    x = tf.keras.layers.Dropout(dropout_rate, name='block1_dropout')(x)

    x = tf.keras.layers.BatchNormalization(
        axis=1,
        name='block1_batch_norm'
    )(x)

    # Bloque 2: TakensConv1D
    takens = TakensConv1D(dx=dx, dy=dy, tau=tau, mu=mu, name='takens_conv1d')(x)
    x_sub, y_minus_1, y_t = takens

    # Proyecciones densas
    x_sub = tf.keras.layers.Dense(dx, activation=None, use_bias=False, name='dense_proj_x')(x_sub)
    y_minus_1 = tf.keras.layers.Dense(dy, activation=None, use_bias=False, name='dense_proj_y_1')(y_minus_1)
    y_t = tf.keras.layers.Dense(1, activation=None, use_bias=False, name='dense_proj_y')(y_t)

    # Kernel layers fijos
    def fixed_kernel(name):
        layer = KernelLayer(
            amplitude=1.0, trainable_amplitude=False,
            length_scale=1.0, trainable_length_scale=False,
            alpha=1.0, trainable_alpha=False,
            kernel_type=kernel_type,
            name=name
        )
        layer.trainable = False
        return layer

    Kx = fixed_kernel('kernel_x')(x_sub)
    Ky1 = fixed_kernel('kernel_y_minus_1')(y_minus_1)
    Ky = fixed_kernel('kernel_y')(y_t)

    # Transferencia de entropía + aplanado
    TE = TransferEntropyLayer(alpha=2, name='transfer_entropy')(Kx, Ky1, Ky)
    flat = RemoveDiagonalFlatten(name='remove_diag_flatten')(TE)

    h = tf.keras.layers.Dense(10, activation='relu', name='dense_1')(flat)
    h = tf.keras.layers.Dropout(dropout_rate, name='dense_dropout')(h)

    out = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(h)

    model = tf.keras.Model(inputs=input_eeg, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            BinaryAccuracy(name='accuracy'),
            F1Score(name='f1_score'),
            Recall(name='sensitivity'),
            Specificity(name='specificity')
        ]
    )
    return model

def build_model(hp):
    # Hiperparámetros a buscar
    dx = hp.Int('dx', min_value=1, max_value=10, step=1)
    dy = hp.Int('dy', min_value=1, max_value=10, step=1)
    tau = hp.Int('tau', min_value=1, max_value=5, step=1)
    mu = hp.Int('mu', min_value=0, max_value=10, step=1)
    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    kernel_type = 'rational_quadratic'
    kernel_size = hp.Int('kernel_size', min_value=3, max_value=125, step=2)
    dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.05)
   

    # Nuevo: rango buscable para kernel_size
    kernel_size = hp.Int(
        'kernel_size',
        min_value=3,      # ventanas muy pequeñas (3 muestras)
        max_value=125,    # hasta ~125 muestras (~0.25 s a 500 Hz)
        step=2            # sólo impares
    )

    # Validación de que el embedding window quepa tras Conv+Pool
    conv_len = 500 - kernel_size + 1
    pool_len = (conv_len - 4) // 4 + 1
    window   = mu + (dx - 1) * tau + 1
    if window > pool_len:
        # Trial inválido: modelo trivial
        m = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        m.compile('adam', 'binary_crossentropy', ['accuracy'])
        return m

    # Construcción del modelo real
    return create_tunable_model(
        dx, dy, tau, mu,
        kernel_type, lr,
        kernel_size,
        dropout_rate
    )

# Configuración del tuner
tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective=kt.Objective("val_accuracy", direction="max"),
    max_trials=70,
    executions_per_trial=2,
    directory="tuner_dir4",
    project_name="takens_te_tuning_4"
)

stop_early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=1
)

# Inicio de la búsqueda
tuner.search(
    x=train_data,
    y=train_labels,
    validation_data=(val_data, val_labels),
    epochs=400,
    callbacks=[stop_early]
)

# Extracción del mejor resultado
best_hp    = tuner.get_best_hyperparameters(num_trials=1)[0]
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
best_score = best_trial.score  # val_f1_score

# Guardado en DataFrame y CSV
import pandas as pd

best_dict = {
    'dx':             best_hp.get('dx'),
    'dy':             best_hp.get('dy'),
    'tau':            best_hp.get('tau'),
    'mu':             best_hp.get('mu'),
    'kernel_size':    best_hp.get('kernel_size'),
    'learning_rate':  best_hp.get('learning_rate'),
    'dropout_rate': best_hp.get('dropout_rate'),
    'val_accuracy':   best_score
}

df_best = pd.DataFrame([best_dict])
print(df_best)
df_best.to_csv('best_hyperparameters_with_score4.csv', index=False)

Trial 70 Complete [00h 01m 09s]
val_accuracy: 0.5769231021404266

Best val_accuracy So Far: 0.7115384638309479
Total elapsed time: 01h 06m 45s
   dx  dy  tau  mu  kernel_size  learning_rate  dropout_rate  val_accuracy
0   6   3    5   5           57         0.0001           0.2      0.711538


In [13]:
# 1) Obtener el modelo ya entrenado con los mejores hiperparámetros
best_model = tuner.get_best_models(num_models=1)[0]

# 2) Guardarlo en /kaggle/working
best_model.save("best_hp_model4.keras") 


# Crear instancia del dataset
db = Dataset_2a('/kaggle/input/dataset-2a')
fs = 250.0 

# Cargar los datos del sujeto en modo 'training'
X, y = load_BCICIV2a(db, sbj=4, mode='evaluation', fs=fs)
print(f'tamaño de X:', X.shape)
print(f'tamaño de X:', y.shape)

results = best_model.evaluate(X, y, return_dict=True)

print("Resultados evaluación:")
print(f"  Accuracy:     {results['accuracy']:.4f}")
print(f"  F1 Score:     {results['f1_score']:.4f}")
print(f"  Loss:         {results['loss']:.4f}")
print(f"  Sensitivity:  {results['sensitivity']:.4f}")
print(f"  Specificity:  {results['specificity']:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


tamaño de X: (116, 22, 500)
tamaño de X: (116,)
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 729ms/step - accuracy: 0.4917 - f1_score: 0.5782 - loss: 0.6973 - sensitivity: 0.8062 - specificity: 0.2539
Resultados evaluación:
  Accuracy:     0.5000
  F1 Score:     0.5972
  Loss:         0.6954
  Sensitivity:  0.7544
  Specificity:  0.2542


In [14]:
# 1) Recuperar los mejores HyperParameters y Trial
best_hp    = tuner.get_best_hyperparameters(num_trials=1)[0]
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

# 2) Extraer puntuación de validación (F1)
best_score = best_trial.score  # corresponde a val_f1_score

# 3) Construir DataFrame con todos los valores
import pandas as pd

best_dict = {
    'dx':             best_hp.get('dx'),
    'dy':             best_hp.get('dy'),
    'tau':            best_hp.get('tau'),
    'mu':             best_hp.get('mu'),
    'kernel_size':    best_hp.get('kernel_size'),
    'learning_rate':  best_hp.get('learning_rate'), 
    'val_accuracy':   best_score
}

df_best = pd.DataFrame([best_dict])
print(df_best)

# 4) Guardar a CSV
df_best.to_csv('best_hyperparameters_with_score4.csv', index=False)

   dx  dy  tau  mu  kernel_size  learning_rate  val_accuracy
0   6   3    5   5           57         0.0001      0.711538


In [15]:
# 1) Recorremos todos los trials guardados en el tuner
records = []
for trial in tuner.oracle.trials.values():
    # Cada trial tiene un objeto HyperParameters donde .values es un dict de {hp_name: value}
    hp_dict = trial.hyperparameters.values.copy()
    # Añadimos el score (val_f1_score) de este trial
    hp_dict['val_accuracy'] = trial.score
    # Opcional: identifica el trial
    hp_dict['trial_id'] = trial.trial_id
    records.append(hp_dict)

# 2) Creamos el DataFrame
df_all = pd.DataFrame(records)

# 3) Ordenamos por la métrica descendente para ver primero los mejores
df_all = df_all.sort_values('val_accuracy', ascending=False).reset_index(drop=True)

# 4) Mostramos y guardamos
print(df_all)
df_all.to_csv('all_trials_hyperparameters4.csv', index=False)

    dx  dy  tau  mu  learning_rate  kernel_size  dropout_rate  val_accuracy  \
0    6   3    5   5         0.0001           57          0.20      0.711538   
1    3   7    3   9         0.0010          101          0.30      0.692308   
2    3   8    4  10         0.0001            7          0.40      0.673077   
3    4   3    2   3         0.0010          101          0.25      0.653846   
4    4   9    5   0         0.0001          109          0.30      0.634615   
..  ..  ..  ...  ..            ...          ...           ...           ...   
65   3  10    2   4         0.0001           81          0.35      0.500000   
66   3   5    4   8         0.0010           93          0.45      0.500000   
67  10   3    4   7         0.0100           79          0.25      0.500000   
68   2   6    1   9         0.0001           93          0.30      0.500000   
69   9   1    4   1         0.0001           61          0.30      0.442308   

   trial_id  
0        28  
1        56  
2        

# Validación cruzada 5 Folds 

In [19]:
# --- Parámetros ---
N_SPLITS     = 5
RANDOM_STATE = 42
EPOCHS       = 1000
FS           = 250.0
MODEL_PATH   = "/kaggle/working/best_hp_model4.keras"

# Métricas a trackear
metrics = ['accuracy', 'specificity', 'sensitivity', 'f1_score']

# Diccionarios para acumular resultados
train_eval = {m: [] for m in metrics}
val_eval   = {m: [] for m in metrics}
test_eval  = {m: [] for m in metrics}

# Callbacks que usamos en cada fold
def get_callbacks(fold):
    return [
        EarlyStopping(
            monitor="val_accuracy",
            mode="max",
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        CSVLogger(f"continued_training_fold{fold}.log", append=True),
        ModelCheckpoint(
            filepath=f"best_model_fold{fold}.keras",
            monitor="val_accuracy",
            mode="max",
            save_best_only=True,
            verbose=1
        ),
    ]

# --- Carga de datos ---
db = Dataset_2a('/kaggle/input/dataset-2a')
X_full, y_full = load_BCICIV2a(db, sbj=5, mode='training',   fs=FS)
X_test, y_test = load_BCICIV2a(db, sbj=5, mode='evaluation', fs=FS)

print(f"Training set:   {X_full.shape}, {y_full.shape}")
print(f"Evaluation set: {X_test.shape},  {y_test.shape}")

# Preparamos el Stratified K-Fold
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_full, y_full), start=1):
    print(f"\n=== Fold {fold}/{N_SPLITS} ===")
    X_train, y_train = X_full[train_idx], y_full[train_idx]
    X_val,   y_val   = X_full[val_idx],   y_full[val_idx]

    # 1) Cargar modelo con pesos y compilación original
    model = load_model(MODEL_PATH, compile=True)

    # 2) Continuar entrenamiento en este fold
    
    model.fit(
        x=X_train, y=y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        callbacks=get_callbacks(fold),
        verbose=2
    )

    # 3) Cargar el mejor modelo guardado por ModelCheckpoint
    best_model = load_model(f"best_model_fold{fold}.keras", compile=True)

    # 4) Evaluar en train, validación y evaluación final
    res_tr   = best_model.evaluate(X_train, y_train, return_dict=True, verbose=0)
    res_va   = best_model.evaluate(X_val,   y_val,   return_dict=True, verbose=0)
    res_test = best_model.evaluate(X_test,  y_test,  return_dict=True, verbose=0)

    # 5) Almacenar cada métrica
    for m in metrics:
        train_eval[m].append( res_tr[m] )
        val_eval[m].append(   res_va[m] )
        test_eval[m].append(  res_test[m] )

# --- Función para mostrar resumen ---
def summarize(eval_dict, title):
    print(f"\n--- {title} ---")
    for m, vals in eval_dict.items():
        mean = np.mean(vals)
        std  = np.std(vals, ddof=1)
        print(f"{m:12s}: {mean:.4f} ± {std:.4f}")

# Imprimimos los promedios ± desviaciones estándar
summarize(train_eval, "TRAIN   (best model por fold)")
summarize(val_eval,   "VALIDATION (best model por fold)")
summarize(test_eval,  "EVALUATION  (best model por fold)")

Training set:   (129, 22, 500), (129,)
Evaluation set: (135, 22, 500),  (135,)

=== Fold 1/5 ===
Epoch 1/1000


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



Epoch 1: val_accuracy improved from -inf to 0.46154, saving model to best_model_fold1.keras
4/4 - 17s - 4s/step - accuracy: 0.5146 - f1_score: 0.5833 - loss: 0.6938 - sensitivity: 0.6604 - specificity: 0.3600 - val_accuracy: 0.4615 - val_f1_score: 0.6111 - val_loss: 0.7001 - val_sensitivity: 0.8462 - val_specificity: 0.0769
Epoch 2/1000

Epoch 2: val_accuracy improved from 0.46154 to 0.53846, saving model to best_model_fold1.keras
4/4 - 0s - 56ms/step - accuracy: 0.5631 - f1_score: 0.6281 - loss: 0.6881 - sensitivity: 0.7170 - specificity: 0.4000 - val_accuracy: 0.5385 - val_f1_score: 0.6842 - val_loss: 0.6995 - val_sensitivity: 1.0000 - val_specificity: 0.0769
Epoch 3/1000

Epoch 3: val_accuracy improved from 0.53846 to 0.57692, saving model to best_model_fold1.keras
4/4 - 0s - 54ms/step - accuracy: 0.6117 - f1_score: 0.6610 - loss: 0.6878 - sensitivity: 0.7358 - specificity: 0.4800 - val_accuracy: 0.5769 - val_f1_score: 0.7027 - val_loss: 0.6996 - val_sensitivity: 1.0000 - val_speci

In [17]:
FS = 250.0 
db = Dataset_2a('/kaggle/input/dataset-2a')
X_full, y_full = load_BCICIV2a(db, sbj=4, mode='training',   fs=FS)
X_test, y_test = load_BCICIV2a(db, sbj=4, mode='evaluation', fs=FS)

In [ ]:
##### import numpy as np
from keras.models import load_model  # o tensorflow.keras

model_paths = [
    "/kaggle/working/best_model_fold1.keras",
    "/kaggle/working/best_model_fold2.keras",
    "/kaggle/working/best_model_fold3.keras",
    "/kaggle/working/best_model_fold4.keras",
    "/kaggle/working/best_model_fold5.keras",
    
]

# Vamos a recoger también las métricas por fold
train_metrics = {}
test_metrics  = {}

print("## Accuracy por fold ##")
for i, path in enumerate(model_paths, start=1):
    model = load_model(path)
    tr = model.evaluate(X_full, y_full, return_dict=True, verbose=0)
    te = model.evaluate(X_test, y_test, return_dict=True, verbose=0)

    # Imprimimos justo aquí la accuracy de este fold
    print(f"Fold {i}: Train ACC = {tr['accuracy']:.4f},  Test ACC = {te['accuracy']:.4f}")

    # Y seguimos acumulando para el resumen
    for key, val in tr.items():
        train_metrics.setdefault(key, []).append(val)
    for key, val in te.items():
        test_metrics.setdefault(key, []).append(val)

# Luego, si quieres, mantienes tu agregado:
print("\n--- Resultados agregados sobre 5 folds ---")
for metric in train_metrics:
    t_arr = np.array(train_metrics[metric])
    e_arr = np.array(test_metrics[metric])
    print(f"{metric:12s} | Train: {t_arr.mean():.4f} ± {t_arr.std():.4f}  |  Test: {e_arr.mean():.4f} ± {e_arr.std():.4f}")

## Accuracy por fold ##


# Std Artículo

In [33]:
import numpy as np

# Datos de validación (Val. Acc) para cada sujeto [9, 8, 3, 7, 1, 5, 6, 4, 2]
val_accuracy = np.array([0.991, 0.985, 0.942, 0.917, 0.899, 0.884, 0.726, 0.723, 0.677])
std_val_acc  = np.array([0.019, 0.020, 0.060, 0.069, 0.083, 0.051, 0.084, 0.137, 0.084])

# Datos de accuracy para cada sujeto
accuracy = np.array([0.869, 0.887, 0.885, 0.611, 0.759, 0.613, 0.565, 0.599, 0.621])
std_acc  = np.array([0.007, 0.023, 0.008, 0.013, 0.010, 0.017, 0.034, 0.049, 0.034])

# Datos de F1-score para cada sujeto
f1_score = np.array([0.877, 0.886, 0.892, 0.603, 0.758, 0.659, 0.553, 0.646, 0.618])
std_f1    = np.array([0.005, 0.024, 0.007, 0.028, 0.011, 0.012, 0.049, 0.026, 0.056])

# Datos de sensitivity para cada sujeto
sensitivity = np.array([0.932, 0.868, 0.929, 0.600, 0.760, 0.775, 0.535, 0.744, 0.625])
std_sen     = np.array([0.007, 0.029, 0.006, 0.057, 0.019, 0.050, 0.079, 0.066, 0.117])

# Datos de specificity para cada sujeto
specificity = np.array([0.806, 0.906, 0.839, 0.623, 0.758, 0.463, 0.596, 0.458, 0.6169])
std_spe     = np.array([0.018, 0.026, 0.017, 0.038, 0.019, 0.070, 0.079, 0.137, 0.133])

# Número de sujetos
n = val_accuracy.size

def mean_and_avg_err(values, errs):
    """
    Calcula la media de `values` y la incertidumbre del promedio
    asumiendo errores independientes:
      sigma_prom = sqrt(sum(err_i^2)) / n
    """
    mean_val = np.mean(values)
    err_avg  = np.sqrt(np.sum(errs**2)) / n
    return mean_val, err_avg

# Calcular media e incertidumbre para cada métrica
m_val,    e_val    = mean_and_avg_err(val_accuracy,  std_val_acc)
m_acc,    e_acc    = mean_and_avg_err(accuracy,      std_acc)
m_f1,     e_f1     = mean_and_avg_err(f1_score,      std_f1)
m_sens,   e_sens   = mean_and_avg_err(sensitivity,   std_sen)
m_spec,   e_spec   = mean_and_avg_err(specificity,   std_spe)

# Mostrar resultados
print(f"Validation ACC Promedio: {m_val:.4f} ± {e_val:.4f}")
print(f"Accuracy Promedio:       {m_acc:.4f} ± {e_acc:.4f}")
print(f"F1 Score Promedio:       {m_f1:.4f} ± {e_f1:.4f}")
print(f"Sensitivity Promedio:    {m_sens:.4f} ± {e_sens:.4f}")
print(f"Specificity Promedio:    {m_spec:.4f} ± {e_spec:.4f}")


Validation ACC Promedio: 0.8604 ± 0.0252
Accuracy Promedio:       0.7121 ± 0.0085
F1 Score Promedio:       0.7213 ± 0.0099
Sensitivity Promedio:    0.7520 ± 0.0197
Specificity Promedio:    0.6740 ± 0.0250
